In [ ]:
%matplotlib inline
!pip install tensorly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 198 kB 5.1 MB/s 
     |████████████████████████████████| 154 kB 13.6 MB/s 



# Basic tensor operations

Example on how to perform basic tensor operations. Specifically unfolding


In [ ]:
import numpy as np
import tensorly as tl
from tensorly.testing import assert_array_equal
from numpy import linalg as LA

The unfolding can be done in different ways. The following is a function that defines the unfolding in the way Dr. Kolda defines it.

In [ ]:
def f_unfold(tensor, mode=0):
    """Unfolds a tensors following the Kolda and Bader definition

        Moves the `mode` axis to the beginning and reshapes in Fortran order
    """
    return np.reshape(np.moveaxis(tensor, mode, 0), 
                      (tensor.shape[mode], -1), order='F')


A tensor is simply a numpy array. Here is a simple example



In [ ]:
rank_3_tensor = tl.tensor(np.arange(60).reshape((3, 4, 5)))
print('* original tensor:\n{}'.format(rank_3_tensor))
print(rank_3_tensor.shape)

* original tensor:
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]
  [30 31 32 33 34]
  [35 36 37 38 39]]

 [[40 41 42 43 44]
  [45 46 47 48 49]
  [50 51 52 53 54]
  [55 56 57 58 59]]]
(3, 4, 5)


**Tesnor unfolding**

We can unfold each tensor using the tensorly unfolding or the definition provided before. After the unfolding is done we can then calculate the gram matrices (X*X')to begin implementing the HOSVD Algorithm as shown below:


In [ ]:
for mode in range(rank_3_tensor.ndim):
  unfolded=tl.unfold(rank_3_tensor, mode)

  print('* mode-{} unfolding:\n{}'.format(mode, unfolded))
  gram=np.matmul(unfolded,np.transpose(unfolded))
  print('* mode-{} gram matrix X Xt:\n{} \n'.format(mode, gram))


* mode-0 unfolding:
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]]
* mode-0 gram matrix X Xt:
[[ 2470  6270 10070]
 [ 6270 18070 29870]
 [10070 29870 49670]] 

* mode-1 unfolding:
[[ 0  1  2  3  4 20 21 22 23 24 40 41 42 43 44]
 [ 5  6  7  8  9 25 26 27 28 29 45 46 47 48 49]
 [10 11 12 13 14 30 31 32 33 34 50 51 52 53 54]
 [15 16 17 18 19 35 36 37 38 39 55 56 57 58 59]]
* mode-1 gram matrix X Xt:
[[11290 12940 14590 16240]
 [12940 14965 16990 19015]
 [14590 16990 19390 21790]
 [16240 19015 21790 24565]] 

* mode-2 unfolding:
[[ 0  5 10 15 20 25 30 35 40 45 50 55]
 [ 1  6 11 16 21 26 31 36 41 46 51 56]
 [ 2  7 12 17 22 27 32 37 42 47 52 57]
 [ 3  8 13 18 23 28 33 38 43 48 53 58]
 [ 4  9 14 19 24 29 34 39 44 49 54 59]]
* mode-2 gram matrix X Xt:
[[12650 12980 13310 13640 13970]
 [12980 13322 13664 14006 14348]
 [13310 13664 14018 14372 14726]
 [13

# HOSVD Algorithm

To implement the algorithm we must:


1.   Define a Tolerance value that we must satisfy. This value will dictate the SNR of the compression, the final size of the core tensor and the compression ratio. 
2.   Calculate the gram matrix for the n-th mode.
3.   Find the eigen vector and the eigen values of each gram matrix
4.   Do a shrink operation to the orginal core tensor with the leading left singular values defined by the tolerance treshold calculated on step 1. 



In [ ]:
# Define Tolerance
tol=0.1
print('tol = ', tol);

## Calculate the treshold: tol^2 ||X||^2 / d

#Square the original tensor
sqr_x=np.square(rank_3_tensor) 

#Calculate norm of the resulting tensor.
normsqr_x=LA.norm(sqr_x) 
print('||X||^2 = ', normsqr_x)

# compute the treshold
eigsum_thresh = np.square(tol) * normsqr_x / 3 
print('eigenvalue sum threshold = tol^2 ||X||^2 / d = {} \n'.format(eigsum_thresh))

np.set_printoptions(suppress=True)
rank=np.zeros(rank_3_tensor.ndim); # vector to store the n-ranks of the tensor
u=[] # List to save each of the factor matrices


#Loop to calculate the gram, eigen and ttm of each mode
for mode in range(rank_3_tensor.ndim):

  unfolded=tl.unfold(rank_3_tensor, mode) #Matrix unfolding
  #print('* mode-{} unfolding:\n{}'.format(mode, unfolded))

  #Right side SVD
  #gram=np.matmul( np.transpose(unfolded),unfolded)

  #left side SVD
  gram=np.matmul(unfolded,np.transpose(unfolded))
  
  #print("unfolded matrix shape: ", unfolded.shape," gram shape: ", gram.shape)
  #print('Eigen Decomposition:')

  #Calculate eigendecomposition of gram matrix where d is the resulting eigenvector and v is the eigenmatrix
  d, v = LA.eig(gram)

  #print("D shape: ", d.shape," V shape: ", v.shape)
  #print('* D-{} \n'.format(d))
  #print('eigen vector: ', list(map('{:.4f}'.format,d)))

  #Reverse sort the resulting eigenvector d
  eigvec = np.msort(d)[::-1]
  pi=np.argsort(d)[::-1] #keep a record of the original position of the values
  
  #print('eigen vector sorted: ', list(map('{:.4f}'.format,eigvec)))
  #Do a cumulative sum on the sorted eigenvector and find the first value that satisfies the intended tolerance
  eigsum = np.cumsum(eigvec[::-1],axis=0)[::-1] 
  print('Reverse cummulative sum of evals of Gram matrix: ', list(map('{:.4f}'.format,eigsum)),'\n')

  rank[mode]=np.count_nonzero(d > eigsum_thresh)
  print('* rank:{:.0f} \n'.format(np.count_nonzero(d > eigsum_thresh)))
  
  #Extract factor matrix by picking out leading eigenvectors of V
  #u= v[:,pi[0:int(rank[mode])]]
  u.append(v[:,pi[0:int(rank[mode])]])
  #print('* mode-{} V:\n{}'.format(mode, v))
  #print('* mode-{} U:\n{}'.format(mode, u))

for i in range(rank_3_tensor.ndim):
  print('factor matrix {} \n'.format(i),u[i])
print('core dimensions: ', rank)

tol =  0.1
||X||^2 =  12211.142370802168
eigenvalue sum threshold = tol^2 ||X||^2 / d = 40.7038079026739 

Reverse cummulative sum of evals of Gram matrix:  ['70210.0000', '457.6188', '-0.0000'] 

* rank:2 

Reverse cummulative sum of evals of Gram matrix:  ['70210.0000', '433.1667', '-0.0000', '-0.0000'] 

* rank:2 

Reverse cummulative sum of evals of Gram matrix:  ['70210.0000+0.0000j', '30.5645+0.0000j', '-0.0000+0.0000j', '-0.0000-0.0000j', '-0.0000+0.0000j'] 

* rank:1 

factor matrix 0 
 [[-0.1736132  -0.89620968]
 [-0.50849656 -0.27343112]
 [-0.84337993  0.34934743]]
factor matrix 1 
 [[-0.39804473  0.73590787]
 [-0.46253128  0.29336806]
 [-0.52701782 -0.14917175]
 [-0.59150437 -0.59171157]]
factor matrix 2 
 [[-0.42434579+0.j]
 [-0.4356371 +0.j]
 [-0.44692842+0.j]
 [-0.45821974+0.j]
 [-0.46951105+0.j]]
core dimensions:  [2. 2. 1.]


Re-folding the tensor :



In [ ]:
#for mode in range(tensor.ndim):
#    unfolding = tl.unfold(tensor, mode)
#    folded = tl.fold(unfolding, mode, tensor.shape)
#    assert_array_equal(folded, tensor)

In [ ]:
rank_3_tensor = tl.tensor([
  [[0,  1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

#print(rank_3_tensor)
print(rank_3_tensor.shape)




(3, 2, 5)


In [ ]:
#Created with matlab using:
#info = create_problem('Type','Tucker','Num_Factors',[3 2 2],'Size',[10 10 2],'Noise',0.01);
#This core should be size [3 2 2] ideally
rank_3_tensor = tl.tensor([
  [[3.2522, -4.1194,  1.2773, -3.4837, -4.4533,  4.7328,  1.3861,  0.4443, -2.1840, -6.1272],
    [-0.4372,  0.4523, -0.1801,  0.3506,  0.6150, -0.5485, -0.2036, -0.0384,  0.2677,  0.8703],
    [0.2775, -3.1669,  0.3043, -3.4528, -0.3776,  2.7690, -0.6667,  0.8877, -0.1961,  0.0847],
    [0.0167,  0.1729, -0.0152,  0.2226, -0.0143, -0.1579,  0.0721, -0.0666,  0.0023, -0.0278],
    [1.0922, -3.4406,  0.5798, -3.5065, -1.4925,  3.3778, -0.0935,  0.7749, -0.7153, -1.6531],
    [0.8745, -1.7865,  0.3781, -1.7098, -1.2048,  1.8367,  0.1772,  0.2986, -0.5721, -1.5406],
    [-0.9146, -0.0432, -0.2478, -0.3934,  1.2757, -0.3010, -0.7167,  0.2394,  0.5652,  1.9942],
    [-2.1805,  4.1096, -0.9373,  3.8902,  2.9985, -4.2962, -0.5381, -0.6754,  1.4527,  3.8672],
    [0.8677, -0.4320,  0.3031, -0.1894, -1.2001,  0.7128,  0.5873, -0.0587, -0.5794, -1.8024],
    [ 2.6731, -2.5217,  0.9804, -1.9389, -3.6152,  3.1858,  1.3655,  0.1010, -1.7614, -5.2084]],
  [[-0.2445, -0.8008, -0.0117, -1.0115,  0.3563,  0.6202, -0.4362,  0.3165,  0.1588,  0.7068],
    [ 1.1509, -1.2514,  0.4269, -1.0304, -1.5275,  1.5087,  0.5467,  0.0851, -0.7525, -2.1983],
    [ 2.1465, -0.7872,  0.6829, -0.1133, -2.9405,  1.4708,  1.4438, -0.2966, -1.4053, -4.4658],
    [-0.7661,  0.8491, -0.2896,  0.7289,  1.0558, -0.9995, -0.3729, -0.0540,  0.5241,  1.4674],
    [ 0.3347,  0.7940,  0.0531,  1.0016, -0.4026, -0.5319,  0.4581, -0.3045, -0.1692, -0.8015],
    [ 0.4765, -0.4281,  0.1920, -0.2992, -0.6741,  0.5294,  0.2539,  0.0084, -0.3225, -0.9158],
    [-0.6519,  2.0948, -0.3479,  2.1176,  0.8676, -1.9890,  0.0600, -0.4751,  0.4440,  0.9244],
    [ 0.0289, -0.3310,  0.0608, -0.3153, -0.0723,  0.3096, -0.0571,  0.0815, -0.0474, -0.0329],
    [-0.2608, -0.5183, -0.0233, -0.6519,  0.3278,  0.3481, -0.3351,  0.2207,  0.1501,  0.6372],
    [-0.0018, -1.5616,  0.0955, -1.7584, -0.0166,  1.3316, -0.4397,  0.4832,  0.0005,  0.3300]],])




In [ ]:
sqr_x=np.square(rank_3_tensor)
normsqr_x=LA.norm(sqr_x)
tol=0.1

eigsum_thresh = np.square(tol) * normsqr_x / 3


print('||X||^2 = ', normsqr_x)
print('tol = ', tol);
print('eigenvalue sum threshold = tol^2 ||X||^2 / d = ', eigsum_thresh)

print('eigenvalue sum threshold = tol^2 ||X||^2 / d = {} \n'.format(eigsum_thresh))
np.set_printoptions(suppress=True)
rank=np.zeros(rank_3_tensor.ndim);
u=[]

for mode in range(rank_3_tensor.ndim):
  unfolded=tl.unfold(rank_3_tensor, mode)
  #print('* mode-{} unfolding:\n{}'.format(mode, unfolded))

  #right side SVD
  #gram=np.matmul( np.transpose(unfolded),unfolded)

  #left side SVD
  gram=np.matmul(unfolded,np.transpose(unfolded))
  
  print("unfolded matrix shape: ", unfolded.shape," gram shape: ", gram.shape)
  print('Eigen Decomposition:')

  d, v = LA.eig(gram)

  print("D shape: ", d.shape," V shape: ", v.shape)
  #print('* D-{} \n'.format(d))
  print('eigen vector: ', list(map('{:.4f}'.format,d)))

  #Reverse sort
  eigvec = np.msort(d)[::-1]
  pi=np.argsort(d)[::-1]
  
  print('eigen vector sorted: ', list(map('{:.4f}'.format,eigvec)))
  eigsum = np.cumsum(eigvec[::-1],axis=0)[::-1]
  print('Reverse cummulative sum of evals of Gram matrix: ', list(map('{:.4f}'.format,eigsum)),'\n')

  rank[mode]=np.count_nonzero(d > eigsum_thresh)
  print('* rank:{:.0f} \n'.format(np.count_nonzero(d > eigsum_thresh)))
  
  #Extract factor matrix by picking out leading eigenvectors of V
  #u= v[:,pi[0:int(rank[mode])]]
  u.append(v[:,pi[0:int(rank[mode])]])
  #print('* mode-{} V:\n{}'.format(mode, v))
  #print('* mode-{} U:\n{}'.format(mode, u))

y=rank_3_tensor
for i in range(rank_3_tensor.ndim):
  print('factor matrix {} \n'.format(i),u[i])
  y = tl.tenalg.mode_dot(y,u[i],i,True)
print('core dimensions: ', rank)
print('core: ', y)

||X||^2 =  81.51622739015059
tol =  0.1
eigenvalue sum threshold = tol^2 ||X||^2 / d =  0.2717207579671687
eigenvalue sum threshold = tol^2 ||X||^2 / d = 0.2717207579671687 

unfolded matrix shape:  (2, 100)  gram shape:  (2, 2)
Eigen Decomposition:
D shape:  (2,)  V shape:  (2, 2)
eigen vector:  ['390.6772', '94.9242']
eigen vector sorted:  ['390.6772', '94.9242']
Reverse cummulative sum of evals of Gram matrix:  ['485.6014', '94.9242'] 

* rank:2 

unfolded matrix shape:  (10, 20)  gram shape:  (10, 10)
Eigen Decomposition:
D shape:  (10,)  V shape:  (10, 10)
eigen vector:  ['361.5971', '78.7988', '45.1758', '0.0095', '0.0070', '0.0010', '0.0015', '0.0026', '0.0039', '0.0043']
eigen vector sorted:  ['361.5971', '78.7988', '45.1758', '0.0095', '0.0070', '0.0043', '0.0039', '0.0026', '0.0015', '0.0010']
Reverse cummulative sum of evals of Gram matrix:  ['485.6014', '124.0043', '45.2055', '0.0297', '0.0202', '0.0132', '0.0089', '0.0050', '0.0024', '0.0010'] 

* rank:3 

unfolded matrix 

In [ ]:
from tensorly.decomposition import tucker
random_state = 12345
tucker_rank = [2, 3, 2]
core, tucker_factors = tucker(rank_3_tensor, rank=tucker_rank, init='random', tol=10e-2, random_state=random_state)
print(core.shape)
print(core,'\n')
print(tucker_factors)

(2, 3, 2)
[[[-18.88954736  -0.71716682]
  [ -0.62703125   4.3966007 ]
  [ -0.46258461  -3.65969945]]

 [[  1.93337048   0.72693804]
  [ -6.20221753   4.53951204]
  [ -4.74713278  -3.00529393]]] 

[array([[-0.99932319, -0.03678541],
       [-0.03678541,  0.99932319]]), array([[-0.59044919, -0.24595615,  0.06556455],
       [ 0.04730365,  0.35932742,  0.35097667],
       [-0.2440164 ,  0.79820934, -0.04164871],
       [ 0.02817768, -0.23463187, -0.21527775],
       [-0.31255822,  0.17064934, -0.43872624],
       [-0.20688132,  0.13602222,  0.01508896],
       [ 0.12352529, -0.05077191, -0.6192479 ],
       [ 0.46727659,  0.03835802,  0.30378395],
       [-0.12176162, -0.16390774,  0.15231531],
       [-0.44706802, -0.19915068,  0.36438515]]), array([[-0.27147609,  0.20202982],
       [ 0.3988059 ,  0.39950253],
       [-0.10959108,  0.03127499],
       [ 0.3548415 ,  0.52078643],
       [ 0.37095423, -0.27421204],
       [-0.44168568, -0.26195878],
       [-0.1000899 ,  0.26736744],
    

In [ ]:
# Decomposition comparison
for i in range(rank_3_tensor.ndim):
  print('factor matrix {} \n'.format(i),u[i])
  print('tensorly: \n',tucker_factors[i],'\n')

print('core{} \n'.format(i),y)
print('tensorly: \n',core,'\n')

factor matrix 0 
 [[ 0.99932295 -0.0367919 ]
 [ 0.0367919   0.99932295]]
tensorly: 
 [[-0.99932319 -0.03678541]
 [-0.03678541  0.99932319]] 

factor matrix 1 
 [[-0.59044849  0.24596273 -0.06556906]
 [ 0.04730401 -0.35932786 -0.3509593 ]
 [-0.24401476 -0.79820606  0.04165746]
 [ 0.02817672  0.23463214  0.21528085]
 [-0.31256144 -0.17063987  0.43873466]
 [-0.20688001 -0.13602913 -0.01509801]
 [ 0.12352468  0.0507919   0.61925813]
 [ 0.46727715 -0.03834735 -0.30377493]
 [-0.12175994  0.16391439 -0.15232208]
 [-0.44706824  0.1991495  -0.36437504]]
tensorly: 
 [[-0.59044919 -0.24595615  0.06556455]
 [ 0.04730365  0.35932742  0.35097667]
 [-0.2440164   0.79820934 -0.04164871]
 [ 0.02817768 -0.23463187 -0.21527775]
 [-0.31255822  0.17064934 -0.43872624]
 [-0.20688132  0.13602222  0.01508896]
 [ 0.12352529 -0.05077191 -0.6192479 ]
 [ 0.46727659  0.03835802  0.30378395]
 [-0.12176162 -0.16390774  0.15231531]
 [-0.44706802 -0.19915068  0.36438515]] 

factor matrix 2 
 [[ 0.27147607 -0.20203304]

In [ ]:
# Tensor reconstruction
x1=y
for i in range(y.ndim):
  x1 = tl.tenalg.mode_dot(x1,u[i],i)
  print('mode: ',i)
  print('updated core shape: ', x1.shape)

print('Reconstructed tensor: ', x1)
print('Original tensor: ', rank_3_tensor)


# Tolerance calculation
diff_sqr = np.square(rank_3_tensor-x1)
diff_normsqr = LA.norm(diff_sqr)
relnorm = np.sqrt(diff_normsqr/normsqr_x);

print('||X-T||/||X|| = ', relnorm, '<= ',tol)

mode:  0
updated core shape:  (2, 3, 2)
mode:  1
updated core shape:  (2, 10, 2)
mode:  2
updated core shape:  (2, 10, 10)
Reconstructed tensor:  [[[ 3.25438004 -4.11135722  1.26539923 -3.49917902 -4.44512072
    4.72693081  1.38527932  0.43126731 -2.15635775 -6.14737412]
  [-0.44288233  0.4530367  -0.1645161   0.35618433  0.60464551
   -0.55295914 -0.21801255 -0.02666906  0.29262564  0.85848239]
  [ 0.26656832 -3.15279907  0.30703404 -3.46083438 -0.37157302
    2.7760627  -0.66658197  0.88225918 -0.19855351  0.07564798]
  [ 0.00405067  0.18786484 -0.01236286  0.21317274 -0.00502082
   -0.1578256   0.05518093 -0.05750337 -0.00118147 -0.04734281]
  [ 1.09720018 -3.45486042  0.57603545 -3.51159513 -1.5041412
    3.34859653 -0.10600525  0.76757993 -0.74311331 -1.64707727]
  [ 0.87932644 -1.78348911  0.39048644 -1.7036309  -1.20284629
    1.84778909  0.18798447  0.31881742 -0.58787995 -1.5226694 ]
  [-0.91915924 -0.05372721 -0.26964967 -0.38115949  1.2522462
   -0.3044239  -0.72779431  0.2

In [ ]:
# Tensor reconstruction

tensorly_reconstruction = tl.tucker_to_tensor((core, tucker_factors))
print('Reconstructed tensor: ', tensorly_reconstruction)
print('Original tensor: ', rank_3_tensor)


# Tolerance calculation
diff_sqr = np.square(rank_3_tensor-tensorly_reconstruction)
diff_normsqr = LA.norm(diff_sqr)
relnorm = np.sqrt(diff_normsqr/normsqr_x);

print('||X-T||/||X|| = ', relnorm, '<= ',tol)

Reconstructed tensor:  [[[ 3.25437189 -4.1113715   1.26540498 -3.49919517 -4.44510821
    4.72695334  1.38527715  0.43128146 -2.15636073 -6.14735686]
  [-0.44288283  0.45305859 -0.16451987  0.356208    0.60464584
   -0.55298193 -0.21800903 -0.02667764  0.2926284   0.85848015]
  [ 0.26658209 -3.1528263   0.30701233 -3.46087221 -0.37159699
    2.77603429 -0.66660612  0.88223673 -0.19853289  0.07564418]
  [ 0.00405518  0.18788177 -0.01236079  0.21319423 -0.00502656
   -0.15783425  0.05519105 -0.05750765 -0.00118639 -0.04735739]
  [ 1.09719671 -3.4548454   0.57601491 -3.5115874  -1.5041398
    3.34854452 -0.10602158  0.76755844 -0.74309129 -1.64705894]
  [ 0.87933591 -1.7835088   0.39048584 -1.70365195 -1.20286011
    1.84779924  0.18798188  0.31881646 -0.58788121 -1.52268168]
  [-0.91915558 -0.05374073 -0.26966186 -0.38117948  1.25223849
   -0.30443982 -0.72780956  0.24357756  0.59959034  1.9861326 ]
  [-2.18597288  4.1108935  -0.94741262  3.87131583  2.98937646
   -4.31968618 -0.55672954